In [ ]:
pip install ultralytics --upgrade


In [1]:
import torch
import torch.nn as nn
import os
import shutil
from pathlib import Path
import json
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import yaml
from ultralytics import YOLO
from torch.utils.data import DataLoader
from torchvision.datasets import VisionDataset
from torchvision.transforms import ToTensor
from tqdm import tqdm
from collections import deque
import albumentations as A


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [7]:

class YOLODatasetOrganizer:
    """
    Organize your dataset into YOLO format structure (train/val split only)
    """
    def __init__(self, base_dir="dataset"):
        self.base_dir = Path(base_dir)
        self.train_dir = self.base_dir / "train"
        self.val_dir = self.base_dir / "val"
        
        # Create directory structure
        for split in ["train", "val"]:
            (self.base_dir / split / "images").mkdir(parents=True, exist_ok=True)
            (self.base_dir / split / "labels").mkdir(parents=True, exist_ok=True)

    def convert_bbox_format(self, bbox, img_width, img_height, format_from="xyxy"):
        if format_from == "xyxy":
            x1, y1, x2, y2 = bbox
            center_x = (x1 + x2) / 2 / img_width
            center_y = (y1 + y2) / 2 / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height
        elif format_from == "xywh":
            x, y, w, h = bbox
            center_x = (x + w/2) / img_width
            center_y = (y + h/2) / img_height
            width = w / img_width
            height = h / img_height
        return center_x, center_y, width, height

    def create_annotation_file(self, annotations, output_path):
        with open(output_path, 'w') as f:
            for ann in annotations:
                class_id, cx, cy, w, h = ann
                f.write(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

    def split_dataset(self, images_dir, annotations_dir, train_ratio=0.8, val_ratio=0.2):
        assert abs(train_ratio + val_ratio - 1.0) < 1e-6, "Ratios must sum to 1.0"

        image_files = list(Path(images_dir).glob("*.jpg")) + \
                     list(Path(images_dir).glob("*.png")) + \
                     list(Path(images_dir).glob("*.jpeg"))
        
        valid_files = []
        for img_file in image_files:
            ann_file = Path(annotations_dir) / f"{img_file.stem}.txt"
            if ann_file.exists():
                valid_files.append(img_file)
        
        print(f"Found {len(valid_files)} valid image-annotation pairs")

        train_files, val_files = train_test_split(valid_files, test_size=val_ratio, random_state=42)

        print(f"Split: Train={len(train_files)}, Val={len(val_files)}")

        splits = {
            "train": (train_files, self.train_dir),
            "val": (val_files, self.val_dir)
        }
        
        for split_name, (files, target_dir) in splits.items():
            for img_file in files:
                # Copy image
                shutil.copy2(img_file, target_dir / "images" / img_file.name)
                # Copy annotation
                ann_file = Path(annotations_dir) / f"{img_file.stem}.txt"
                if ann_file.exists():
                    shutil.copy2(ann_file, target_dir / "labels" / ann_file.name)
        return len(train_files), len(val_files)

    def copy_test_set(self, test_images_dir, test_labels_dir):
        test_dir = self.base_dir / "test"
        (test_dir / "images").mkdir(parents=True, exist_ok=True)
        (test_dir / "labels").mkdir(parents=True, exist_ok=True)
        image_files = list(Path(test_images_dir).glob("*.jpg")) + \
                      list(Path(test_images_dir).glob("*.png")) + \
                      list(Path(test_images_dir).glob("*.jpeg"))
        for img_file in image_files:
            ann_file = Path(test_labels_dir) / f"{img_file.stem}.txt"
            shutil.copy2(img_file, test_dir / "images" / img_file.name)
            if ann_file.exists():
                shutil.copy2(ann_file, test_dir / "labels" / ann_file.name)
        print(f"Copied {len(image_files)} test images and annotations")
        return len(image_files)

    def create_dataset_yaml(self, save_path="dataset.yaml", nc=2, names=['cheating', 'not_cheating'], include_test=True):
        config = {
            'path': str(self.base_dir.absolute()),
            'train': 'train/images',
            'val': 'val/images',
            'nc': nc,
            'names': names
        }
        if include_test:
            config['test'] = 'test/images'
        with open(save_path, 'w') as f:
            yaml.dump(config, f, default_flow_style=False)
        print(f"Dataset configuration saved to {save_path}")
        return save_path

In [8]:
class DataAugmentationPipeline:
    def __init__(self):
        import albumentations as A
        from albumentations.pytorch import ToTensorV2
        
        self.transform = A.Compose([
            # Spatial transforms
            A.HorizontalFlip(p=0.5),
            A.RandomRotate90(p=0.3),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
            
            # Color transforms
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=20, p=0.5),
            A.CLAHE(clip_limit=2.0, p=0.3),
            
            # Noise and blur
            A.OneOf([
                A.MotionBlur(blur_limit=3),
                A.GaussianBlur(blur_limit=3),
                A.MedianBlur(blur_limit=3),
            ], p=0.3),
            
            A.OneOf([
                A.GaussNoise(var_limit=(10, 50)),
                A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5)),
            ], p=0.3),
            
            # Weather effects
            A.OneOf([
                A.RandomShadow(p=0.3),
                A.RandomSunFlare(p=0.1),
            ], p=0.2),
            
            # Final resize
            A.Resize(640, 640),
        ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

In [14]:
class TrainingConfig:
    """
    Comprehensive training configuration
    """
    def __init__(self):
        self.config = {
            # Model settings
            'model_size': 's',  # n, s, m, l, x
            'img_size': 640,
            'num_classes': 2,
            
            # Training parameters
            'epochs': 50,
            'batch_size': 16,
            'learning_rate': 0.01,
            'weight_decay': 0.0005,
            'momentum': 0.937,
            'patience':10,
            
            # Optimization
            'optimizer': 'AdamW',  # SGD, AdamW, Adam
            'lr_scheduler': 'cosine',  # linear, cosine
            
            # Data augmentation
            'hsv_h': 0.015,
            'hsv_s': 0.7,
            'hsv_v': 0.4,
            'degrees': 10.0,
            'translate': 0.1,
            'scale': 0.5,
            'shear': 0.0,
            'perspective': 0.0,
            'flipud': 0.0,
            'fliplr': 0.5,
            'mosaic': 1.0,
            'mixup': 0.1,
            'copy_paste': 0.1,
            
            # Loss weights
            'box_loss_gain': 7.5,
            'cls_loss_gain': 0.5,
            'dfl_loss_gain': 1.5,
            
            # Validation
            'val_period': 1,
            'save_period': 10,
            
            # Device
            'device': 'cuda' if torch.cuda.is_available() else 'cpu',
            'workers': 8,
            
            # Paths
            'project': 'runs/detect',
            'name': 'cheating_detection',
        }
    
    def save_config(self, path="training_config.yaml"):
        """Save configuration to file"""
        with open(path, 'w') as f:
            yaml.dump(self.config, f, default_flow_style=False)
        print(f"Training configuration saved to {path}")
    
    def load_config(self, path):
        """Load configuration from file"""
        with open(path, 'r') as f:
            self.config = yaml.safe_load(f)
        print(f"Training configuration loaded from {path}")


In [15]:
def setup_complete_pipeline():
    """
    Complete setup pipeline for YOLO cheating detection
    """
    print("Setting up YOLO Cheating Detection Pipeline...")
    
    # 1. Create dataset organizer
    organizer = YOLODatasetOrganizer("cheating_dataset")
    
    # 2. Create training configuration
    config = TrainingConfig()
    config.save_config("cheating_detection_config.yaml")
    
    # 3. Create dataset YAML
    dataset_yaml = organizer.create_dataset_yaml("cheating_dataset.yaml")
    
    print("\nSetup completed! Next steps:")
    print("1. Place your images in a single directory")
    print("2. Create YOLO format annotations (.txt files)")
    print("3. Use organizer.split_dataset() to organize your data")
    print("4. Run training with the generated configuration")
    
    return organizer, config, dataset_yaml


In [ ]:
organizer, config, dataset_yaml = setup_complete_pipeline()
    
organizer = YOLODatasetOrganizer("cheating_dataset")

# Split train/val from all images+labels (customize ratios as needed)
organizer.split_dataset(
    images_dir="/kaggle/input/aggregate-dataset/Aggregate Dataset/Train/images",
    annotations_dir="/kaggle/input/aggregate-dataset/Aggregate Dataset/Train/labels",
    train_ratio=0.8,
    val_ratio=0.2
)

# Copy test set
organizer.copy_test_set(
    test_images_dir="/kaggle/input/primary-test/Primary_test/Images",
    test_labels_dir="/kaggle/input/primary-test/Primary_test/Labels"
)

# Create dataset yaml
organizer.create_dataset_yaml("cheating_dataset.yaml")

model = YOLO('yolov8s.pt')  # or yolov8m.pt/yolov8l.pt for larger

results = model.train(
    data='cheating_dataset.yaml',
    epochs=50,
    imgsz=320,
    batch=16,
    lr0=0.01,
    weight_decay=0.0005,
    momentum=0.937,
    patience=10,
    save_period=10,
    project='runs/detect',
    name='cheating_detection_Aggregeto',
    device='cuda' if torch.cuda.is_available() else 'cpu',
    verbose=True,
    plots=True,
    val=True,     
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.1,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    
)

Setting up YOLO Cheating Detection Pipeline...
Training configuration saved to cheating_detection_config.yaml
Dataset configuration saved to cheating_dataset.yaml

Setup completed! Next steps:
1. Place your images in a single directory
2. Create YOLO format annotations (.txt files)
3. Use organizer.split_dataset() to organize your data
4. Run training with the generated configuration
Found 5036 valid image-annotation pairs
Split: Train=4028, Val=1008
Copied 35 test images and annotations
Dataset configuration saved to cheating_dataset.yaml
Ultralytics 8.3.160 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=cheating_dataset.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, d

train: Scanning /kaggle/working/cheating_dataset/train/labels.cache... 4096 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4096/4096 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 235.8±89.5 MB/s, size: 32.1 KB)


val: Scanning /kaggle/working/cheating_dataset/val/labels.cache... 1072 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1072/1072 [00:00<?, ?it/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      47/50       1.8G      1.174      1.013      1.271         57        320: 100%|██████████| 256/256 [00:33<00:00,  7.56it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:08<00:00,  4.13it/s]

                   all       1072       3379        0.5      0.696      0.621      0.375



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      48/50       1.8G      1.171     0.9979      1.256         51        320:  21%|██        | 53/256 [00:07<00:25,  7.97it/s]

In [ ]:
def export_and_zip_yolo_model(
    weights_path,
    dataset_yaml_path,
    export_formats=["onnx"],  # e.g., ["onnx", "torchscript"]
    export_dir="exported_model",
    zip_name="yolo_cheating_detection_export.zip"
):

    os.makedirs(export_dir, exist_ok=True)

    # 1. Copy weights (.pt)
    pt_name = os.path.basename(weights_path)
    pt_export_path = os.path.join(export_dir, pt_name)
    shutil.copy2(weights_path, pt_export_path)

    # 2. Copy dataset YAML
    yaml_name = os.path.basename(dataset_yaml_path)
    yaml_export_path = os.path.join(export_dir, yaml_name)
    shutil.copy2(dataset_yaml_path, yaml_export_path)

    # 3. Export to other formats (e.g., ONNX, TorchScript)
    model = YOLO(weights_path)
    exported_files = []
    for fmt in export_formats:
        result = model.export(format=fmt, dynamic=True)
        # result will hold the export path, copy it to export_dir
        export_file = result[0] if isinstance(result, (list, tuple)) else result
        if os.path.isfile(export_file):
            shutil.copy2(export_file, export_dir)
            exported_files.append(os.path.join(export_dir, os.path.basename(export_file)))
        else:
            # Sometimes model.export() saves to cwd; check for files with known extensions
            for file in os.listdir('.'):
                if file.endswith(f'.{fmt}'):
                    shutil.copy2(file, export_dir)
                    exported_files.append(os.path.join(export_dir, file))

    # 4. Zip everything
    zip_fp = shutil.make_archive(zip_name.replace('.zip', ''), 'zip', export_dir)

    print(f"Exported and zipped model to: {zip_fp}")
    print("Contents zipped:")
    for f in [pt_export_path, yaml_export_path] + exported_files:
        print(" -", os.path.basename(f))
    return zip_fp

In [ ]:
export_and_zip_yolo_model(
    "/kaggle/working/runs/detect/cheating_detection_v12/weights/best.pt",
    "/kaggle/working/cheating_dataset.yaml"
)

In [ ]:

# def predict_and_zip(input_dir, model_path="runs/detect/cheating_detection_Correcto/weights/best.pt", output_dir="outputs", zip_name="outputs.zip"):
#     # Clean up previous outputs
#     if os.path.exists(output_dir):
#         shutil.rmtree(output_dir)
#     os.makedirs(output_dir, exist_ok=True)

#     # Load YOLOv8 model (class names will be loaded from the .pt file)
#     model = YOLO(model_path)

#     # Predict on input directory
#     results = model.predict(
#         source=input_dir,
#         save=True,         # Save images with bounding boxes
#         save_txt=True,     # Save label files
#         project=output_dir,
#         name="predict",
#         exist_ok=True,
#         imgsz=640,
#         conf=0.25
#     )

#     # Zip up the output images + labels
#     prediction_path = os.path.join(output_dir, "predict")
#     with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
#         for root, _, files in os.walk(prediction_path):
#             for file in files:
#                 file_path = os.path.join(root, file)
#                 arcname = os.path.relpath(file_path, prediction_path)
#                 zipf.write(file_path, arcname)

#     print(f"\n✅ Predictions (with class names) saved to {prediction_path}")
#     print(f"📦 Zipped into: {zip_name}")


# predict_and_zip(input_dir="/kaggle/input/corrected-primary-dataset/PRIMARY Dataset/Evaluation")



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/62 /kaggle/input/corrected-primary-dataset/PRIMARY Dataset/Evaluation/IMG_3911.JPG: 640x480 2 not_cheatings, 41.2ms
image 2/62 /kaggle/input/corrected-primary-dataset/PRIMARY Dataset/Evaluation/IMG_3912.JPG: 640x480 2 not_cheatings, 6.9ms
image 3/62 /kaggle/input/corrected-primary-dataset/PRIMARY Dataset/Evaluation/IMG_3913.JPG: 640x480 2 not_cheatings, 2 cheatings, 6.9ms
image 4/62 /kaggle/input/corrected-primary-dataset/PRIMARY Dataset/Evalua